<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/CU-30u4v4d_Analiza-wizualizacja/colabs/Wizualizacje_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 711, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 711 (delta 118), reused 59 (delta 48), pack-reused 516
Receiving objects: 100% (711/711), 5.77 MiB | 26.14 MiB/s, done.
Resolving deltas: 100% (358/358), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [6]:
!git checkout Refactoring

Branch 'Refactoring' set up to track remote branch 'Refactoring' from 'origin'.
Switched to a new branch 'Refactoring'


In [7]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 200 kB 32.6 MB/s 
     |████████████████████████████████| 199 kB 13.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0a1475f2d0aa5e1fe51af3e14097837a6098a6f3729dd27e056e735f5edf21e0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-13 19:42:45--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [8]:
from lib.pyspark_init import create_spark_context, add_kaggle_data
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

In [9]:
spark = create_spark_context()
spark

In [10]:
%%time
data = spark.read.parquet(JOINED_DATA)
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|nm0000017|[tt0021156, tt001...|[movie, movie, mo...|[Morocco, Gefahre...|[0, 0, 0, 0, 0, 0...|     1919|     \N|[Drama,Romance, C...| actress|[["Mademoiselle A...|    Marlene Dietrich|tt0051201,tt00211...|
|nm0000051|[tt0028417, tt002...|[movie, movie, mo...|[Troubled Waters,...|[0, 0, 0, 0, 0, 0...|     1935|     \N|[Crime,Mystery, C...|   actor|[["John Merriman"...|        

In [12]:
data.createOrReplaceTempView('data')

In [14]:
spark.sql('select* from data').show()
spark.sql("""select min(startYear), max(endYear) from data""").show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|nm0000017|[tt0021156, tt001...|[movie, movie, mo...|[Morocco, Gefahre...|[0, 0, 0, 0, 0, 0...|     1919|     \N|[Drama,Romance, C...| actress|[["Mademoiselle A...|    Marlene Dietrich|tt0051201,tt00211...|
|nm0000051|[tt0028417, tt002...|[movie, movie, mo...|[Troubled Waters,...|[0, 0, 0, 0, 0, 0...|     1935|     \N|[Crime,Mystery, C...|   actor|[["John Merriman"...|        

In [16]:
spark.sql("""select* from data where startYear = 0004""").show()

+------+------+---------+-------------+-------+---------+-------+------+--------+----------+-----------+--------------+
|nconst|tconst|titleType|originalTitle|isAdult|startYear|endYear|genres|category|characters|primaryName|knownForTitles|
+------+------+---------+-------------+-------+---------+-------+------+--------+----------+-----------+--------------+
+------+------+---------+-------------+-------+---------+-------+------+--------+----------+-----------+--------------+



In [17]:
Appearances = spark.sql("""SELECT (LENGTH(knownForTitles) - LENGTH(REPLACE(knownForTitles,",","")) + 1) AS CounterknownForTitles, count(*) as ilu_aktorow from data group by CounterknownForTitles""").toPandas()
Appearances

,CounterknownForTitles,ilu_aktorow
0,1,893831
1,3,167390
2,5,6
3,4,732232
4,2,269695


In [19]:
Null_counter_table = spark.sql("""SELECT count(*) as nulls_category from data where category is null""").toPandas()
Null_counter_table

,nulls_category
0,0
